# Save and retrieve jobs

Quantum workflows often take a while to complete and can run over many sessions. Restarting your Python kernel means you'll lose any results stored in memory. To avoid loss of data, you can save results to file and retrieve results of past jobs from IBM Quantum&trade; so your next session can continue where you left off.

## Retrieve jobs from IBM Quantum

IBM Quantum automatically stores results from every job for you to retrieve at a later date. Use this feature to continue quantum programs across kernel restarts and review past results. You can get the ID of a job programmatically through its `job_id` method, or you can see all your submitted jobs and their IDs through the [Jobs dashboard](https://quantum.ibm.com/jobs).

To find a job programatically, use the [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) method. The following cell finds any jobs submitted in the last day. The `created_after` argument must be a [`datetime.datetime`](https://docs.python.org/3.8/library/datetime.html#datetime.datetime) object.

In [1]:
import datetime
from qiskit_ibm_runtime import QiskitRuntimeService

one_day_ago = datetime.datetime.now() - datetime.timedelta(days=1)

service = QiskitRuntimeService()
service.jobs(created_after=one_day_ago)

[<RuntimeJob('cogiau7imm49f6et38a0', 'sampler')>,
 <RuntimeJob('crgja91qzd5000886v60', 'sampler')>,
 <RuntimeJob('cogi9gftcirtooeunfe0', 'sampler')>,
 <RuntimeJob('cogi8qqai5rk2r0u9bu0', 'sampler')>]

You can also select by backend, job state, session, and more. For more information, see [`QiskitRuntimeService.jobs`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#jobs) in the API documentation.

Once you have the job ID, use the [`QiskitRuntimeService.job`](/api/qiskit-ibm-runtime/qiskit_ibm_runtime.QiskitRuntimeService#job) method to retrieve it.

In [ ]:
# Get ID of last job for demonstration
job_id = service.jobs()[0].job_id()
print(job_id)

In [2]:
retrieved_job = service.job(job_id)
retrieved_job.result()

[<RuntimeJob('cogiau7imm49f6et38a0', 'sampler')>,
 <RuntimeJob('crgja91qzd5000886v60', 'sampler')>,
 <RuntimeJob('cogi9gftcirtooeunfe0', 'sampler')>,
 <RuntimeJob('cogi8qqai5rk2r0u9bu0', 'sampler')>]

## Save results to disk

You may also want to save results to disk. To do this, use Python's built-in JSON library with Qiskit IBM Runtime's encoders.

In [3]:
import json
from qiskit_ibm_runtime import RuntimeEncoder
with open("result.json", "w") as file:
    json.dump(retrieved_job.result(), file, cls=RuntimeEncoder)

You can then load this array from disk in a separate kernel.

In [4]:
from qiskit_ibm_runtime import RuntimeDecoder
with open("result.json", "r") as file:
    result = json.load(file, cls=RuntimeDecoder)

result[0].data.meas.array  # access measurement data

array([[1],
       [3],
       [3],
       [2],
       [2],
       [1],
       [2],
       [3],
       [0],
       [3]], dtype=uint8)

In [5]:
# Cleanup the file we created (this cell should be hidden from the user)
import pathlib
pathlib.Path('result.json').unlink()